In [1]:
import pandas as pd
import numpy as np
import annoy
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.applications import vgg16
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import keras
from keras.layers import Dense, Dropout, Flatten
from keras import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers import GlobalMaxPooling2D
from PIL import Image
import os
from sklearn.metrics.pairwise import cosine_similarity
import cv2
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelBinarizer
from keras.applications.xception import Xception,preprocess_input
from keras.preprocessing import image
from keras.layers import Input
from keras.backend import reshape
from sklearn.neighbors import NearestNeighbors
from tensorflow.keras.models import Sequential,load_model,Model
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Dropout,Flatten,Reshape,UpSampling2D,Input,Lambda,Conv2DTranspose


In [ ]:
images=pd.read_csv('../input/my-data/images_list.csv')
images=images.drop('Unnamed: 0',axis=1)
images.rename(columns={'0':'img_id'},inplace=True)
print(images.shape)
images.head()

In [ ]:
images['path']=images['img_id'].apply(lambda x: f"../input/h-and-m-personalized-fashion-recommendations/images/{x[0:3]}/{x}")

In [ ]:
images.iloc[0,1]

# Image similarity model

In [ ]:
imgs_model_width, imgs_model_height = 224, 224
nb_closest_images = 5 

In [ ]:
vgg_model = vgg16.VGG16(weights='imagenet')

# remove the last layers in order to get features instead of predictions
feat_extractor = Model(inputs=vgg_model.input, outputs=vgg_model.get_layer("fc2").output)
top_model = feat_extractor.output
top_model = Dense(4096, activation='relu')(top_model)
top_model = Dense(1072, activation='relu')(top_model)
output_layer=Dense(600,activation='relu')(top_model)
final_model=Model(inputs=vgg_model.input,outputs=output_layer)
# print the layers of the CNN
final_model.summary()

In [ ]:
importedImages = []

In [ ]:
for f in tqdm(images['path'][0:5000]):
    filename =f
    original = load_img(filename, target_size=(224, 224))
    numpy_image = img_to_array(original)
    image_batch = np.expand_dims(numpy_image, axis=0)
    importedImages.append(image_batch)
    
#processed_imgs = preprocess_input(images.copy())

In [ ]:
imagess = np.vstack(importedImages)
imagess.shape

In [ ]:
processed_imgs=preprocess_input(imagess)

In [ ]:
imgs_features = feat_extractor.predict(processed_imgs)

print("features successfully extracted!")
imgs_features.shape

In [ ]:
imgs_features

In [ ]:
cosSimilarities = cosine_similarity(imgs_features)
cos_similarities_df = pd.DataFrame(cosSimilarities, columns=images['path'][0:5000], index=images['path'][0:5000])
cos_similarities_df.head()

In [ ]:
cos_similarities_df.to_csv('similarities.csv')